In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import testsets
import evaluation

from os.path import normpath
import os

# TODO: load training data
FILE_NAME = "./twitter-dev-data.txt"

if os.path.isfile(FILE_NAME):
    print("The file has been found.")
else:
    raise IOError("File not found.")

The file has been found.


# Corpus 

Corpus object to read the data and preprocess it

In [ ]:
import json
import itertools

from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize

class Corpus:
    
    def __init__(self, file_path):
        # Private vars
        self._corpus_loaded = False
        self._corpus_parsed = False
        self._sentence_tokenized = False
        
        self._document_view_pointer = 0
        
        self.file_path = file_path 
        # Checks if the file exists
        if os.path.isfile(FILE_PATH):
            print("The file has been found.")
        else:
            raise IOError("File not found.")
        
        self.raw_docs = []
        self.json_docs = []
        
        # Tokenized sentences
        self.sent_tokens = []
        # Flat list of setences
        self.flat_sent_tokens = []
        
    
    def load_corpus(self):
        with open(self.file_path, "r") as corpus:
            for json_line in corpus:
                self.raw_docs.append(json_line)
        
        self._corpus_loaded = True
    
    
    def parse_corpus(self):
        for json_line in self.raw_docs:
            parsed_json = json.loads(json_line)
            self.json_docs.append(parsed_json)
        
        self._corpus_parsed = True
    
    
    def tokenize_sentences(self):

        for article in self.json_docs:
            content = article["content"]
            sent_tokenized = sent_tokenize(content)
            self.sent_tokens.append(sent_tokenized)
            
        self.flat_sent_tokens = list(itertools.chain.from_iterable(self.sent_tokens))
        
    
    def print_summary_of_corpus(self):
        if not self._corpus_loaded:
            self.load_corpus()
        
        if not self._corpus_parsed:
            self.parse_corpus()
            
        if not self._sentence_tokenized:
            self.tokenize_sentences()
        
        print("Number of documents: {}.".format(str(len(self.json_docs))))
        print("Number of sentences: {}.".format(str(len(self.flat_sent_tokens))))
    
    
    def print_document(self, doc_idx):
        print("-- Printing document with index: {} --".format(doc_idx))
        print(self.json_docs[doc_idx])

    
    def get_document(self, doc_idx):
        return self.json_docs[doc_idx]
    
    
    def get_next_document(self):        
        self._document_view_pointer += 1
        if self._document_view_pointer > len(self.json_docs):
            return None
        else:
            return self.json_docs[(self._document_view_pointer - 1)]
        
    
    def set_current_document(self, json_doc):
        self.json_docs[(self._document_view_pointer - 1)] = json_doc
    
    
    def reset_pointer(self):
        self._document_view_pointer = 0
        
    
    def set_pointer(self, pointer):
        self._document_view_pointer = pointer        


In [2]:



for classifier in ['myclassifier1', 'myclassifier2', 'myclassifier3']: # You may rename the names of the classifiers to something more descriptive
    if classifier == 'myclassifier1':
        print('Training ' + classifier)
        # TODO: extract features for training classifier1
        # TODO: train sentiment classifier1
    elif classifier == 'myclassifier2':
        print('Training ' + classifier)
        # TODO: extract features for training classifier2
        # TODO: train sentiment classifier2
    elif classifier == 'myclassifier3':
        print('Training ' + classifier)
        # TODO: extract features for training classifier3
        # TODO: train sentiment classifier3

    for testset in testsets.testsets:
        # TODO: classify tweets in test set

        predictions = {'163361196206957578': 'neutral', '768006053969268950': 'neutral', '742616104384772304': 'neutral', '102313285628711403': 'neutral', '653274888624828198': 'neutral'} # TODO: Remove this line, 'predictions' should be populated with the outputs of your classifier
        evaluation.evaluate(predictions, testset, classifier)

        evaluation.confusion(predictions, testset, classifier)


Training myclassifier1


FileNotFoundError: [Errno 2] No such file or directory: 'twitter-test1.txt'